In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time
import re
import jieba

In [2]:
# 数据整合，输出新数据
def merge_file(file1,file2):
    feature = pd.read_csv(file1,sep=",")
    label = pd.read_csv(file2,sep=",")
    data = feature.merge(label,on='id') # 按照id列整合两个训练集表
    data["X"] = data[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1) # "title","content"两列合为一列"X"
    df = data[['id', 'title', 'content', 'label', 'X']]
    return df


data = merge_file("Train_DataSet.csv","Train_DataSet_Label.csv")

In [3]:
# 停用词加载和清洗
def stopwords_list(filepath):
    stopwords=[line.strip() for line in open(filepath,'r',encoding='utf-8').readlines()]
    return stopwords

stopwords=stopwords_list('stopwords.txt')

def stop_clean(strs):
    strs = [w for w in strs if not w in stopwords]
    return strs

# 只保留汉字
def character_save(strs):
    strs = re.findall(r'[\u4e00-\u9fa5]',strs)
    return strs

In [4]:
#处理训练集，将训练集的文本信息和label信息合并，清洗特殊符合，同时将文本内容进行分词
def data_pre_handle(df):
    # 1.去空值
    dataDropNa=df.dropna(axis=0, how='any')  
#    dfDropNa["X"]=dataDropNa["X"].apply(lambda x: str(x).replace("\\n","").replace(".","").replace("\n","").replace("　","").replace("↓","").replace("/","").replace("|","").replace(" ",""))
    dataDropNa["X_split"]=dataDropNa["X"]
    dataDropNa["X_split"]=dataDropNa["X_split"].apply(lambda x:"".join(character_save(x)))
    dataDropNa["X_split"]=dataDropNa["X_split"].apply(lambda x:"".join(stop_clean(x)))
    dataDropNa["X_split"]=dataDropNa["X_split"].apply(lambda x:" ".join(jieba.cut(x)))
    
    

    return dataDropNa
 
data = data_pre_handle(data)

C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dongy\AppData\Local\Temp\jieba.cache
Loading model cost 0.613 seconds.
Prefix dict has been built succesfully.
C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:8: SettingW

In [5]:
data["X_split"][0:]

0       问责 领导 上 黄镇 党委书记 张涛 宣国 才 真 一手遮天   天 看  人 举报 施  ...
1       江歌 事件 教会 孩子 善良  时 更 懂 保护  去 一年  江歌 悲剧  日 再次 刷 ...
2       绝味 鸭 脖 广告 开 黄腔 引 众怒 双  拼值  双 亿  销售额  中国  全世界 感...
3       央视 曝光 东 一 医药企业  槽罐车 改成 垃圾车 夜间 偷排 高浓度 废水 年  东 高...
4       恶劣 极 央视 都 曝光  南通 东 一 医药企业  槽罐车 改成 洒水车 夜间 偷排 高浓...
5       央视 曝光 南通 一 医药企业 夜间 偷排 高浓度 废水 丢脸 昨晚 央视 一套 晚间新闻 ...
6       粉丝 爆料 五洲 国际 无锡 项目 涉嫌 诈骗 非法 集资  金融街 号   非法 集资 诈...
7       年内 约  锂电 重组 失败 资  高 估值 收购 说 不 摘  中国 电池 联盟  数 显...
8       男子 梦想 一夜 暴富 持 水泥块 砸机 一分钱 都 没取  近日 江苏 扬州 谢  盗取 ...
9       北京 家 法院 供暖 纠纷案件 主体  供暖费 追缴 山海 网 北京   冬天   年 更 ...
10      手机号 开头  注意 看 完   冷汗 都 出  搜 搜狐 原 标题 手机号 开头  注意 ...
11      网红 土坯房 书记 落马  背后 原 标题 网红 土坯房 书记 落马  背后 深耕 卢氏 官...
12      曾  品质 烤肉 两年    难吃  三鼎 甲 曾  尔滨 烤肉 界 位 显赫 两年 前去 ...
13      土俄 军火交易 再 波折 鱼熊 无法 兼顾 土  成美 俄 交锋 牺牲品 搜狐 军事 搜狐网...
14        一  闲鱼  网上交易 平台 购买  一 代付款 服务    国外 网站 上 购物   ...
15      标题 不 签字 不 意 不 反抗  些 人扮 普天庆  样子 好   新兴 商铺 拆迁  态...
16      伙   已 说  教育 废品 文 不 作文 一篇 文章 写  写   不 知道  跑   天...
17      车震 视频 

In [6]:
# #处理训练集，将训练集的文本信息和label信息合并，清洗特殊符合，同时将文本内容进行分词
# def merge_feature_label(file1,file2):
#     feature=pd.read_csv(file1,sep=",")
#     label=pd.read_csv(file2,sep=",")
#     data=feature.merge(label,on='id') # 按照id列整合两个训练集表
#     data["X"]=data[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1) # 两列合为一列
#     dataDropNa=data.dropna(axis=0, how='any')  # 去空值
#     dataDropNa["X"]=dataDropNa["X"].apply(lambda x: str(x).replace("\\n","").replace(".","").replace("\n","").replace("　","").replace("↓","").replace("/","").replace("|","").replace(" ",""))
#     dataDropNa["X_split"]=dataDropNa["X"].apply(lambda x:" ".join(jieba.cut(x)))
#     return dataDropNa
 
# dataDropNa=merge_feature_label("Train_DataSet.csv","Train_DataSet_Label.csv")

In [7]:
# def clean_chineses_text(text):
#     """
#     中文数据清洗  stopwords_chineses.txt存放在博客园文件中
#     :param text:
#     :return:
#     """
#     text = BeautifulSoup(text, 'html.parser').get_text() #去掉html标签
#     text =jieba.lcut(text);
#     stopwords = {}.fromkeys([line.rstrip() for line in open('stopwords.txt')]) #加载停用词(中文)
#     eng_stopwords = set(stopwords) #去掉重复的词
#     words = [w for w in text if w not in eng_stopwords] #去除文本中的停用词
#     return ' '.join(words)

In [8]:
#处理测试数据
def process_test(test_name):
    test=pd.read_csv(test_name,sep=",")
    test["X"]=test[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1)
    test["X_split"]=test["X"].apply(lambda x:"".join(character_save(x)))
    test["X_split"]=test["X_split"].apply(lambda x:" ".join(jieba.cut(x)))
    test["X_split"]=test["X_split"].apply(lambda x:"".join(stop_clean(x)))
    
    return test
 
testData=process_test("Test_DataSet.csv")

In [11]:
#获取文本内容的tf-idf表示
xTrain=data["X_split"]
xTest=testData["X_split"]
# TfidfVectorizer生成词向量
vec = TfidfVectorizer(ngram_range=(1,2),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
# Fit_transform学习到一个字典，并返回Document-term的矩阵(即词典中的词在该文档中出现的频次)
xTrain_tfidf = vec.fit_transform(xTrain)  # 先拟合数据，然后转化它将其转化为标准形式，一般应用在训练集
xTest_tfidf = vec.transform(xTest)  # 找中心和缩放等实现标准化，一般用在测试集
yTrain=data["label"]  # y值

#训练逻辑回归模型
clf = LogisticRegression(C=4, dual=True)
clf.fit(xTrain_tfidf, yTrain)

C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


LogisticRegression(C=4, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
#预测测试集，并生成结果提交
preds=clf.predict_proba(xTest_tfidf)


In [16]:
type(preds)

numpy.ndarray

In [17]:
preds=np.argmax(preds,axis=1) # 当axis=1时，表示返回行方向上数值最大值下标


In [19]:
preds.shape

(7356,)

In [20]:
test_pred=pd.DataFrame(preds)


In [22]:
type(test_pred)

pandas.core.frame.DataFrame

In [23]:
test_pred.shape

(7356, 1)

In [24]:
test_pred.columns=["label"]
print(test_pred.shape)
test_pred["id"]=list(testData["id"])
test_pred[["id","label"]].to_csv('sub_lr_baseline3.csv',index=None)

(7356, 1)


In [27]:
#训练支持向量机模型
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.fit(xTrain_tfidf, yTrain)
 
#预测测试集，并生成结果提交
preds=lin_clf.predict(xTest_tfidf)


In [29]:
type(preds)

numpy.ndarray

In [30]:
preds.shape

(7356,)

In [26]:
test_pred=pd.DataFrame(preds)


In [32]:
type(test_pred)

pandas.core.frame.DataFrame

In [33]:
test_pred.shape

(7356, 1)

In [34]:
test_pred.columns=["label"]
test_pred["id"]=list(testData["id"])
test_pred[["id","label"]].to_csv('sub_svm_baseline3.csv',index=None)